In [1]:
# Packages loading

import pandas as pd
import datetime
import matplotlib.pyplot as plt
from datetime import date
import hashlib
import random, string

In [2]:
# Import data and target
df = pd.read_csv(r'C:\Users\jay_g\Imperial College London\CDM Group 5 - WP - Documents\General\customer_information.csv') #change to own source's directory 

In [3]:
# convert just colums "weight" and "height" into numeric.
df["weight"] = pd.to_numeric(df["weight"])
df["height"] = pd.to_numeric(df["height"])
df['heightsquared']=df['height']**2

#df.insert(1,"BMI",[])
df['bmi']=df['weight']/df['heightsquared']

# delete the column 'weight','height'and 'heightsquared'
del df['weight']
del df['height']
del df['heightsquared']

In [4]:
# Generate a function to convert birthdate to age group
# Years of birth
birthdate=pd.to_datetime(df.birthdate)
df['birth_year'] = pd.DatetimeIndex(birthdate).year

# Age
def from_birthdate_to_age(birth_date):
    now=pd.Timestamp('now')
    now_year,now_month,now_day = now.year, now.month, now.day
    birth_date = pd.to_datetime(birth_date)
    birth_year, birth_month, birth_day = birth_date.year, birth_date.month, birth_date.day
    age = now_year - birth_year
    if now_month >= birth_month:
        if now_day >= birth_day:
            age = now_year - birth_year + 1
    return (age)

df['age'] = df['birthdate'].apply(from_birthdate_to_age)

# Banding age to groups
bins_age = [0, 20, 30, 40, 50, 60, 80]
labels_age= ['<20','20-30','30-40','40-50','50-60','60-70']
df['age_groups'] = pd.cut(df.age, bins = bins_age, labels = labels_age)

In [5]:
# Banding smoking per week
## we are defining smoking <=40 cig per week as light smokers, 40-175 as intermediate smokers, >175 as heavy smokers
### https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2865193/
#### https://pubmed.ncbi.nlm.nih.gov/1614993/
bins_smok= [0, 40, 175, 500]
labels_smok=['light smokers','intermediate smokers','heavy smokers']
df['smoking_status'] = pd.cut(df.avg_n_cigret_per_week, bins = bins_smok, labels = labels_smok)

# Banding avg_n_drinks_per_week
## 0-3.9 as low-level alc consumption, 4-6.9 as moderate alc consumption,7-10 as high-level alc consumption
bins_alc = [0,4,7,10]
label_alc = ['low', 'moderate', 'high']
df['level of drinking_status'] = pd.cut(df.avg_n_drinks_per_week, bins = bins_alc, labels = label_alc, right = False)

In [11]:
#SHA ----------------------------------------------------------------

#Salt generator
def randomword(length):
   letters = string.ascii_lowercase #generates lowercase letters
   return ''.join(random.choice(letters) for i in range(length)) #generates salt from letters

df['NI_enc']=df['national_insurance_number'].str.encode('utf-8') #utf encoding needed for sha function

key='password123'.encode('utf-8') #encoding key

#Hash function
hashes=[]
salt=[]
for i in range(len(df['NI_enc'])):
    salt.append(randomword(10).encode('utf-8'))
    hashes.append(hashlib.sha1(key+salt[i]+df['NI_enc'][i]).hexdigest()) #hash function applied 

df['hash']=hashes
df['postcode_split'] = df['postcode'].str.split().str[0] 
hash_cols=['given_name','surname','phone_number','national_insurance_number','blood_group','postcode','birthdate','age','bank_account_number','birth_year','avg_n_drinks_per_week','avg_n_cigret_per_week']

#Lookup table generation
secure_df=df[hash_cols]
secure_df['hash']=df['hash']
secure_df['salt']=salt
new_cust=df.drop(columns=hash_cols)
new_cust=new_cust.drop(columns='NI_enc')

C:\Users\jay_g\AppData\Local\Temp\ipykernel_25428\369217079.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secure_df['hash']=df['hash']
C:\Users\jay_g\AppData\Local\Temp\ipykernel_25428\369217079.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secure_df['salt']=salt
